In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Analyzing Soccer players").getOrCreate()

24/04/09 17:48:07 WARN Utils: Your hostname, Abhijits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.105 instead (on interface en0)
24/04/09 17:48:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/09 17:48:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/04/09 17:48:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
## Player details

player = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("/Volumes/T7/GettingStartedSpark2/player.csv")

In [7]:
player.show(5)

+---+-------------+------------------+------------------+-------------------+------+------+
| id|player_api_id|       player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+------------------+------------------+-------------------+------+------+
|  1|       505942|Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|   Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|       Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|     Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|      Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
+---+-------------+------------------+------------------+-------------------+------+------+
only showing top 5 rows



In [9]:
player.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)



In [10]:
### Player Attributes

player_attributes = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("/Volumes/T7/GettingStartedSpark2/Player_Attributes.csv")

In [11]:
player_attributes.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- attacking_work_rate: string (nullable = true)
 |-- defensive_work_rate: string (nullable = true)
 |-- crossing: string (nullable = true)
 |-- finishing: string (nullable = true)
 |-- heading_accuracy: string (nullable = true)
 |-- short_passing: string (nullable = true)
 |-- volleys: string (nullable = true)
 |-- dribbling: string (nullable = true)
 |-- curve: string (nullable = true)
 |-- free_kick_accuracy: string (nullable = true)
 |-- long_passing: string (nullable = true)
 |-- ball_control: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- sprint_speed: string (nullable = true)
 |-- agility: string (nullable = true)
 |-- reactions: string (nullable = true

In [12]:
### Checking if the details are of same player:

player_attributes.select("player_api_id")\
        .distinct()\
            .count()

11060

In [13]:
player.select("player_api_id")\
        .distinct()\
            .count()

11060

In [14]:
player = player.drop('id', 'player_fifa_api_id')
player.columns

['player_api_id', 'player_name', 'birthday', 'height', 'weight']

In [15]:
player_attributes = player_attributes.drop('id', 
                     'player_fifa_api_id',
                     'preferred_foot',
                     'attacking_work_rate',
                     'defensive_work_rate',
                     'crossing',
                     'jumping',
                     'sprint_speed',
                     'balance',
                     'aggression',
                     'short_passing','potential')
player_attributes.columns

['player_api_id',
 'date',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

### User Defined functions -> UDF

In [17]:
### Apart from the functions present in the spark.sql.functions it lets us create our own user defined functions (udf)

In [18]:
from pyspark.sql.functions import udf

In [19]:
year_extract_udf = udf(lambda date: str(date).split('-')[0])    ### Defining a function

player_attributes = player_attributes.withColumn(
    "year",
    year_extract_udf(player_attributes.date)    ### Using func
)

In [20]:
player_attributes = player_attributes.drop("date")

In [21]:
pa_2016  = player_attributes.filter(player_attributes.year == 2016)

In [23]:
### Unique players

pa_striker_2016 = pa_2016.groupBy("player_api_id")\
    .agg({"finishing": "avg", "shot_power": "avg", "acceleration": "avg"})
    
### Using multiple aggregation at the same time

In [24]:
pa_striker_2016.show(5)

+-------------+-----------------+-----------------+---------------+
|player_api_id|   avg(finishing)|avg(acceleration)|avg(shot_power)|
+-------------+-----------------+-----------------+---------------+
|       309726|75.44444444444444|74.11111111111111|           76.0|
|        26112|             53.0|             51.0|           76.0|
|        38433|            68.25|             74.0|           74.0|
|        38576|             61.0|             59.0|           74.0|
|       438780|             49.0|             78.0|           57.0|
+-------------+-----------------+-----------------+---------------+
only showing top 5 rows



In [26]:
pa_striker_2016 = pa_striker_2016.withColumnRenamed("avg(finishing)", "finishing")\
    .withColumnRenamed("avg(acceleration)", "acceleration")\
    .withColumnRenamed("avg(shot_power)", "shot_power")

In [28]:
### Weight of each prop for selection

weight_finishing = 1
weight_shot_power = 2
weight_acceleration = 1

total_weight = weight_finishing +weight_shot_power + weight_acceleration

In [29]:
### Grading strikers

strikers = pa_striker_2016.withColumn("striker_grade", (pa_striker_2016.finishing * weight_finishing + \
                                       pa_striker_2016.acceleration * weight_acceleration + \
                                       pa_striker_2016.shot_power * weight_shot_power) / total_weight)

In [31]:
strikers = strikers.drop("acceleration", "shot_power", "finishing")

In [32]:
strikers.show(5)

+-------------+-----------------+
|player_api_id|    striker_grade|
+-------------+-----------------+
|       309726|75.38888888888889|
|        26112|             64.0|
|        38433|          72.5625|
|        38576|             67.0|
|       438780|            60.25|
+-------------+-----------------+
only showing top 5 rows



In [33]:
### sorting and filtering

strikers = strikers.filter(strikers.striker_grade > 70) \
    .sort(strikers.striker_grade.desc())
    
strikers.show(5)

+-------------+-------------+
|player_api_id|striker_grade|
+-------------+-------------+
|        20276|        89.25|
|        37412|         89.0|
|        38817|        88.75|
|        32118|        88.25|
|        31921|         87.0|
+-------------+-------------+
only showing top 5 rows



In [36]:
#### Join 

#striker_details = player.join(strikers, player.player_api_id == strikers.player_api_id)

## OR

striker_details = player.join(strikers, ["player_api_id"])

In [37]:
striker_details.columns

['player_api_id',
 'player_name',
 'birthday',
 'height',
 'weight',
 'striker_grade']

## Broadcast Variables

In [38]:
### Perform join operations by broadcasting dataframes

In [40]:
from pyspark.sql.functions import broadcast

### Now the broadcast variables are stored in the cache, so we can not save too large data to broadcast

### As strikers df is smaller, we broadcast that

In [41]:
striker_details = player.select("player_api_id", "player_name").join(broadcast(strikers), ["player_api_id"], 'inner')

## Accumulators

In [42]:
### Analyzing heading accuracy with height

In [45]:
player_heading_acc = player_attributes.select("player_api_id", "heading_accuracy").join(
    broadcast(player),
    ["player_api_id"]
)

In [46]:
player_heading_acc.columns

['player_api_id',
 'heading_accuracy',
 'player_name',
 'birthday',
 'height',
 'weight']

In [47]:
### Now we bucket the players according to height

### Defining accumulator to maintain count

short_count = spark.sparkContext.accumulator(0)
medium_low_count = spark.sparkContext.accumulator(0)
medium_high_count = spark.sparkContext.accumulator(0)
tall_count = spark.sparkContext.accumulator(0)

In [48]:
def count_players_by_height(row):
    height = float(row["height"])
    
    if height <= 175:
        short_count.add(1)
    elif height > 175 and height <=183:
        medium_low_count.add(1)        ## Adding to accumulator
    elif height > 183 and height <=195:
        medium_high_count.add(1)
    elif height > 195:
        tall_count.add(1)

In [50]:
player_heading_acc.foreach(lambda x : count_players_by_height(x))

## Foreach method 

In [51]:
all_players  = [short_count.value, medium_low_count.value, medium_high_count.value, tall_count.value]
all_players 

[38408, 197916, 124822, 6810]

## Storing data as CSV and JSON

In [52]:
pa_2016.select("player_api_id", "overall_rating")\
    .coalesce(1)\
    .write\
    .option("header", "true")\
    .csv("players_overall.csv")
    
    #### coalesce joins the data distributed across all workers to one

## Custom Accumulators

In [ ]:
from pyspark.accumulators import  AccumulatorParam

class VectorAccumulatorParam(AccumulatorParam):
    def zero(self, value):
       return [0.0] * len(value)
    def addInPlace(self, val1, val2):
        for i in range(len(val1)):
            val1[i] += val2[i]
            return val1
va = spark.sparkContext.accumulator([1.0, 2.0, 3.0], VectorAccumulatorParam())
